In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
df = pd.read_csv('../data/raw_postpro.csv', encoding='cp949')
y = df["key2"]
df['key2'] = df['key2'].astype('category')

In [44]:
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)

In [6]:
# df['key2'].mode()
# grouped_df = df.groupby('key2')
# grouped_df.head()
# grouped_df[['Subject','Machinery', 'Assembly', '청구품목']].head()
# series1 = grouped_df['청구서번호'].count()
# series1.iloc[:,]

In [7]:

# plt.bar(grouped_df['청구서번호'].count().index,grouped_df['청구서번호'].count())
# plt.xlabel("key2")
# plt.ylabel('count')
# plt.xticks(rotation=45)
# plt.ylim(500,4200)
# # 특정 품목만 많이 청구됨

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   청구서번호        20517 non-null  object  
 1   No.          20517 non-null  int64   
 2   Subject      20517 non-null  object  
 3   Machinery    20517 non-null  object  
 4   Assembly     20517 non-null  object  
 5   청구품목         20517 non-null  object  
 6   Part No.1    20517 non-null  object  
 7   Part No.2    20517 non-null  object  
 8   key1         20517 non-null  object  
 9   key2         20517 non-null  category
 10  리드타임_음수제거    20517 non-null  int64   
 11  청구량          20517 non-null  int64   
 12  견적           20517 non-null  object  
 13  견적수량         20517 non-null  int64   
 14  견적화폐         20517 non-null  object  
 15  견적단가         20517 non-null  int64   
 16  발주번호         20517 non-null  object  
 17  발주처          20517 non-null  object  
 18  발주           20517 non-nul

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le_x = preprocessing.LabelEncoder()
# df = df.apply(le.fit_transform)

X = df[['Subject','Machinery', 'Assembly' , "Part No.1","청구품목"]]
X = X.apply(le_x.fit_transform)

y = df["key2"]
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)
# y_encoded = le_y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [17]:
my_list = [i for i in range(61)]
print(my_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


In [18]:
my_list = le_y.inverse_transform(my_list)

In [19]:
my_list

array(['ADAPTER', 'ANODES', 'ARM', 'BATTERY', 'BEARING', 'BELT', 'BINO',
       'BLOCK', 'BOLT', 'BRAKE', 'BUSHING', 'CARD', 'CARTRIDGE', 'CLAMP',
       'CLIP', 'COMPRESSOR', 'CONNECTING ROD', 'CONTROL', 'COOLER',
       'COVER', 'CYLINDER', 'DAMPER', 'DISC', 'FILTER', 'GASKET', 'GAUGE',
       'GEAR', 'GENERATOR', 'HARNESS', 'HOSE', 'INJECTOR', 'KEY', 'LINE',
       'LINK', 'LIQUID', 'MANIFOLD', 'MONITOR', 'MOTOR', 'NET', 'NUT',
       'O-RING', 'PIN', 'PIPE', 'PISTON', 'PLATE', 'PUMP', 'SCREW',
       'SEAT', 'SHAFT', 'SPACER', 'SPRING', 'SWITCH', 'TOOL',
       'TRANSFORMER', 'TUBE', 'TURBINE', 'UNIT', 'VALVE', 'WASHER',
       'WINCH', 'WIRE'], dtype=object)

In [27]:
list1 = [9,15,28,52,59]
list1 = le_y.inverse_transform(list1)

In [74]:
list1

array(['BRAKE', 'COMPRESSOR', 'HARNESS', 'TOOL', 'WINCH'], dtype=object)

In [46]:
values_to_exclude = ['BRAKE', 'COMPRESSOR', 'HARNESS', 'TOOL', 'WINCH']
df = df[~df['key2'].isin(values_to_exclude)]

In [75]:
df

,청구서번호,No.,Subject,Machinery,Assembly,청구품목,Part No.1,Part No.2,key1,key2,...,창고입고수량,Control No.,입고창고,창고출고,창고출고수량,출고선박,출고운반선,선박입고,선박입고수량,완료 여부
0,ELS-BS-ESP-2004002,12,NO.1 A/E C18 MAJOR O/H PARTS,NO.1 GENERATOR ENGINE,323-6480 LINES GP-FUEL,SEAL-O-RING-STOR,7.00E-275,0,7.00E-275,COOLER,...,1,혜인 발주,BS,2020-09-19,1,BLO,본선 선적,0,0,0
1,OCA-BS-ESP-2010004,5,운전시간 대비 #2 AUX ENG TOP END O/H PARTS,NO.2 GENERATOR ENGINE,GASKET KIT,OIL COOLER & LINES,7.00E-275,0,7.00E-275,COOLER,...,1,혜인 발주,BS,2020-09-18,1,JBG,본선 선적,0,0,0
2,MIR-BS-ESP-2011002,119,운전시간 대비 #2 AUX ENG(CAT 3512C) MAJOR O/H PARTS,NO.2 GENERATOR ENGINE,285-8374 MANIFOLD GP-EXH,WASHER,7.00E-275,0,7.00E-275,COOLER,...,1,0,BS,2020-12-12,1,BON,SANWA FONTAINE,0,0,0
3,BLO-BS-ESP-2003002,16,#1 AUX ENG TOP END O/H PARTS,NO.1 GENERATOR ENGINE,159-8828 TURBO GP,BOLT-HIGH TEMP,7.00E-275,0,7.00E-275,COOLER,...,1,혜인 발주,BS,2021-06-16,1,MIR,체항수리 시 사용,0,0,0
4,DEO-BS-ESP-2105008,97,DEO-E-210512-01,NO.1 GENERATOR ENGINE,2N4727 INSTRUMNT PANEL GP,SEAL,7.00E-275,0,7.00E-275,COOLER,...,1,0,BS,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20512,BLO-BS-ESP-2006006,1,"BLO-F-ENGINE-200515-03 BOAT DAVIT CYL 외, AIRCO...",HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_NET BOAT DAVIT,φ150 × φ85 × 695st,FF-SE-1CB150BB695ABD-YP2J,φ150 × φ85 × 695stFF-SE-1CB150BB695ABD-YP2J,CYLINDER,...,2,20-09-161,BW,0,0,0,0,0,0,0
20513,BLO-BS-ESP-2004008,1,BLO-F-ENGINE-200403-01 (NAVI DECK CRANE DERRIN...,HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_N.D CRANE_BOOM_LIFTER,φ160 × φ112 × 1130st,S-2CA160A20.6N1130-1,φ160 × φ112 × 1130stS-2CA160A20.6N1130-1,CYLINDER,...,1,20-07-114,BW,2020-08-26,1,BLO,BLO 상가시 선적,0,0,0
20514,ELS-BS-ESP-1908002,2,ELS-F-ENG-190628-02,6M NET BOAT,ELECTRIC CLUTCH,PULLY,Φ305X5V-3,0,Φ305X5V-3,BELT,...,1,19-08-164,BW,2019-09-04,1,ELS,ONENESS,2019-10-02,1,완료
20515,BLO-BS-ESP-1906014,2,BLO-F-ENGINE-190617-01 (NET BOAT PARTS),7M NET BOAT,HAND HYDRAULIC STEERING SYSTEM,CYLINDER,Φ50 x S195 CYLINDER,0,Φ50 x S195 CYLINDER,CYLINDER,...,1,19-07-134,BW,2019-08-20,1,BLO,세이셀 컨테이너,2019-10-12,1,완료


In [82]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le_x = preprocessing.LabelEncoder()
# df = df.apply(le.fit_transform)

X = df[['Subject','Machinery', 'Assembly' , "Part No.1","청구품목"]]
X = X.apply(le_x.fit_transform)

y = df["key2"]
le_y = preprocessing.LabelEncoder()
y = le_y.fit_transform(y)

# y_encoded = le_y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.1)

In [77]:
X.iloc[15:21,:]

,Subject,Machinery,Assembly,Part No.1,청구품목
15,411,25,1370,4741,5387
16,411,25,1370,5046,4800
17,411,25,1370,5662,5387
18,1249,145,1230,1671,1854
19,976,145,1230,1671,1842
20,1249,145,1230,1800,2491


In [78]:
y

array([16, 16, 16, ...,  5, 18, 18])

# CategoricalNB

In [9]:
import numpy as np
from sklearn.naive_bayes import CategoricalNB
 
rng = np.random.RandomState(100)
clf = CategoricalNB(alpha=0.1).fit(X_train, y_train) 
## alpha : Laplace Smoothing 파라미터

In [10]:
y_pred = clf.predict(X_train)
y_pred

array([20, 33, 45, ..., 40, 50, 57])

In [11]:
print(clf.get_params()) ## CategoricalNB 클래스 인자 설정 정보
print('정확도 : ', clf.score(X_train,y_train)) ## 성능 평가 점수(Accuracy
print('정확도 : ', clf.score(X_test,y_test)) ## 성능 평가 점수(Accuracy

{'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': 'warn', 'min_categories': None}
정확도 :  0.9631389752025833
정확도 :  0.7570662768031189


In [12]:
clf.score(X_train, y_train)

0.9631389752025833

In [13]:
from sklearn.metrics import f1_score
# 모델 평가
from sklearn.metrics import mean_squared_error, r2_score
y_pred = clf.predict(X)
mse = mean_squared_error(y, y_pred)


# 모델 예측
y_pred = clf.predict(X_test)

# mse 값 계산
print("MSE: ", mse)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)


MSE:  41.45021201930107
F1-score: 0.7604636520475917


# OneVsRestClassifier

# LogisticRegression

## Tree

In [15]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# df = pd.read_csv('../data/raw_postpro.csv')

df = df.apply(preprocessing.LabelEncoder().fit_transform)
X = df[['Machinery', 'Assembly' , "Part No.1","청구품목"]]
y = df["key2"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [16]:
tree = DecisionTreeClassifier(max_depth=24, random_state= 0) # 트리 생성
tree.fit(X_train, y_train)
print("훈련 세트 정확도 : {:.3f}".format(tree.score(X_train,y_train)))
print("테스트 세트 정확도 : {:.3f}".format(tree.score(X_test,y_test))) 

훈련 세트 정확도 : 0.974
테스트 세트 정확도 : 0.755


In [17]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = tree.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)


F1-score: 0.7540515071387015


## y_train 값에 가중치 딕셔너리

In [18]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
type(y_train)

pandas.core.series.Series

In [20]:
# 클래스 레이블의 빈도수 계산
class_counts = np.bincount(y_train)

# 클래스 가중치 계산
total_samples = len(y_train)
class_weights = total_samples / (len(class_counts) * class_counts)

# 클래스와 가중치를 매칭하여 딕셔너리 생성
class_weight_dict = dict(enumerate(class_weights))

print(class_weight_dict)

{0: 2.118626565121983, 1: 10.348675914249684, 2: 4.4844262295081965, 3: 8.6795346377578, 4: 0.2826319052211048, 5: 5.724799441925358, 6: 4.076751117734724, 7: 2.8322691975841243, 8: 0.348079655589252, 9: 20.697351828499368, 10: 1.3189488910318226, 11: 29.896174863387976, 12: 1.8303780528604885, 13: 2.7455670792907325, 14: 9.609484777517565, 15: 9.965391621129326, 16: 3.057563338301043, 17: 2.023049426845803, 18: 0.8434657484968395, 19: 0.8998848621086682, 20: 0.6777470372052691, 21: 15.827386692381872, 22: 2.468491502481576, 23: 0.9474139921496191, 24: 0.08387330853194881, 25: 0.41522465088038857, 26: 0.7643908345752608, 27: 7.474043715846994, 28: 3.3633196721311474, 29: 2.0079520430633715, 30: 1.1305276208844193, 31: 4.804742388758783, 32: 12.230253353204173, 33: 3.68582977767797, 34: 1.6307004470938897, 35: 4.204149590163935, 36: 6.4063231850117095, 37: 1.1597654041831542, 38: 3.7896559685984763, 39: 1.084941829719725, 40: 0.15939903659389326, 41: 1.582738669238187, 42: 1.35891703924

## Xgboost : softmax

In [83]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 테스트 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# xgboost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

# 모델 학습
model.fit(X_train, y_train)

# 예측 결과 출력
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8532608695652174


In [84]:
from sklearn.metrics import f1_score


# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

F1-score: 0.8514914549978977


In [85]:
# 정확도, 정밀도, 재현율, F1 점수를 계산합니다.
print(classification_report(y_test, y_pred))

# 혼동 행렬을 출력합니다.
# print(confusion_matrix(y_test, y_pred)[:10])
# for i in confusion_matrix(y_test, y_pred):
    # print(i)

              precision    recall  f1-score   support

           0       0.75      0.33      0.46        18
           1       1.00      1.00      1.00         1
           2       1.00      0.33      0.50         3
           3       1.00      1.00      1.00         4
           4       0.93      0.90      0.91       106
           5       1.00      0.56      0.71         9
           6       1.00      1.00      1.00        10
           7       1.00      1.00      1.00         9
           8       0.87      0.88      0.87        88
           9       0.89      0.92      0.91        26
          10       1.00      1.00      1.00         1
          11       0.88      0.82      0.85        17
          12       0.78      0.88      0.82         8
          13       1.00      1.00      1.00         4
          14       1.00      0.87      0.93        15
          15       0.60      0.60      0.60        10
          16       0.83      0.91      0.87        33
          17       0.77    

D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
import xgboost as xgb
#XGBoost DMatrix 생성
dtrain = xgb.DMatrix(X_train, label=y_train)

# XGBoost 모델 파라미터 설정
params = {
    "objective": "multi:softmax",  # 다중 분류를 위한 손실 함수
    "num_class": 61,  # 클래스의 개수
    "eval_metric": "merror",  # 분류 정확도 평가 지표
    "random_state": 42
}
# 모델 학습
xgb_model = xgb.train(params, dtrain)

# 예측을 위한 DMatrix 생성
# X_test = ...
dtest = xgb.DMatrix(X_test)

# 예측 수행
y_pred = xgb_model.predict(dtest)

In [23]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = model.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

F1-score: 0.843450153794426


In [24]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
# 텍스트 데이터
df = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
X = list(df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1'])
y = df["key2"]

In [31]:
# CountVectorizer를 이용하여 feature 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [32]:
# y 값을 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [33]:
unique_sorted_array = np.sort(np.unique(y))

# 결과 출력
print(unique_sorted_array)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60]


In [34]:
y

array([18, 18, 18, ...,  5, 20, 20])

In [66]:
type(X)

pandas.core.frame.DataFrame

In [84]:
# 희소 행렬을 밀집 행렬로 변환
# dense_matrix = X.toarray()

# for i in dense_matrix[0]:
#     if i != 0 :
#         print(i)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression(random_state=42)

param_grid = {
      'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000]
}

grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1_macro', return_train_score=True)

grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
print("최적의 하이퍼파라미터:", best_params)

In [ ]:
best_model = grid_search.best_estimator_
print("최적의 모델:", best_model)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("정확도:", accuracy)
print("정밀도:", precision)
print("재현율:", recall)
print("1 점수:", f1)

In [35]:
# 학습 데이터와 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# xgboost 모델 생성
model = XGBClassifier(objective='multi:softmax', num_class=61)

# 모델 학습
model.fit(X_train, y_train)

# 예측 결과 출력
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8581871345029239


In [69]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))


In [37]:
 print_metrics(y_test, X_test, model)

Recall: 0.8235
Accuracy: 0.8582
Precision: 0.8675
F1-score: 0.8404


In [9]:
# import tensorflow as tf
# print(tf.__version__)

2.12.0


In [38]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression 모델 생성 및 학습
# lr = LogisticRegression(random_state=42, C=7, solver="liblinear", penalty='l1', multi_class="ovr", max_iter=1000)
lr = LogisticRegression(random_state=42, C=14, solver="liblinear", penalty='l1', multi_class= "ovr", max_iter=1000,class_weight='balanced')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)


In [39]:
y_pred = lr.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average ='macro')
recall = recall_score(y_test, y_pred, average ='macro')
f1 = f1_score(y_test, y_pred, average ='macro')

print("정확도:", accuracy)
print("정밀도:", precision)
print("재현율:", recall)
print("1 점수:", f1)

정확도: 0.9035087719298246
정밀도: 0.9126387544327941
재현율: 0.8758071244631589
1 점수: 0.8863699874063273


# svc model

In [82]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', decision_function_shape='ovr')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [83]:
svc_pred = svc.predict(X_test)
svc_acc = accuracy_score(y_test, svc_pred)
print_metrics(y_test, X_test, svc)

Recall: 0.8271
Accuracy: 0.8667
Precision: 0.8836
F1-score: 0.8449


# RandomForestClassifier

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
y_train,X_train

(array([23,  8, 54, ..., 37, 37, 53]),
        Subject  Machinery  Assembly  Part No.1  청구품목
 11959     1619        111        72       3598   691
 10414      232          1        96       2963  5065
 4059       897         38       869        425  5785
 19277      835        134       702       7298  4566
 11939      361        106      1163       3597   998
 ...        ...        ...       ...        ...   ...
 11667      533        106       271       3472  4729
 16253      449        106      1356       5629  2363
 14675      909        106       156       5008  4729
 14729     1574        113       299       5016  4728
 11725      458        106       489       3488  2523
 
 [16191 rows x 5 columns])

In [68]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [70]:
rf_pred = rf.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)
print_metrics(y_test, X_test, rf)

Recall: 0.7468
Accuracy: 0.7910
Precision: 0.7901
F1-score: 0.7611


# DecisionTreeClassifier

In [71]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [72]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [83]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10),
    'min_samples_split': range(2, 10),
    'min_samples_leaf': range(1, 10)
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("최적의 모델:", best_model)

최적의 모델: DecisionTreeClassifier(max_depth=9, random_state=42)


In [73]:
y_pred = dt.predict(X_test)

# 정확도, 정밀도, 재현율, F1 점수를 계산합니다.
print(classification_report(y_test, y_pred))

# 혼동 행렬을 출력합니다.
print(confusion_matrix(y_test, y_pred).shape)
# for i in confusion_matrix(y_test, y_pred):
#     print(i)

              precision    recall  f1-score   support

           0       0.58      0.45      0.51        33
           1       0.50      0.75      0.60         4
           2       0.50      0.41      0.45        17
           3       1.00      0.80      0.89         5
           4       0.88      0.82      0.85       221
           5       0.71      0.59      0.65        17
           6       0.86      0.75      0.80        16
           7       0.73      0.70      0.71        23
           8       0.73      0.74      0.73       177
           9       0.91      0.85      0.88        48
          10       0.50      1.00      0.67         2
          11       0.42      0.48      0.45        29
          12       0.78      0.78      0.78        23
          13       1.00      1.00      1.00         7
          14       0.71      0.85      0.77        20
          15       0.57      0.63      0.60        19
          16       0.58      0.69      0.63        71
          17       0.74    

In [90]:
# 텍스트 데이터
import pandas as pd
df = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
X = list(df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1'])
y = df["key2"]

In [91]:
# CountVectorizer를 이용하여 feature 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)


In [92]:
# y 값을 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [93]:
# 학습 데이터와 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [94]:
from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], # 트리의 최대 깊이
#     'min_samples_split': [2, 3, 4, 5], # 노드를 분할하기 위한 최소 샘플 수
#     'min_samples_leaf': [1, 2, 3, 4, 5] # 리프 노드에 있어야 하는 최소 샘플 수
# }
param_grid = {
    # 'max_depth': [None] + (range(3, 10)),  
    'min_samples_split': [2, 3, 4, 5], # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 3, 4, 5], # 리프 노드에 있어야 하는 최소 샘플 수
    # 'n_estimators': [50, 100, 200, 300],
    # 'learning_rate': [0.01, 0.1, 0.2, 0.5],
    'max_depth':[3, 4, 5, 6, 7, 8, 9, 10]
}

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', return_train_score=True)


In [96]:
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5]},
             return_train_score=True, scoring='accuracy')

In [97]:
best_params = grid_search.best_params_
print(best_params)

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [98]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [99]:
y_pred = best_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
# 정확도, 정밀도, 재현율, F1 점수를 계산합니다.
print(classification_report(y_test, y_pred))

# 혼동 행렬을 출력합니다.
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           1       0.00      0.00      0.00         7
           2       0.20      0.06      0.09        17
           3       0.00      0.00      0.00        10
           4       0.85      0.76      0.80       215
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        13
           7       0.00      0.00      0.00        23
           8       0.77      0.53      0.62       175
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00        50
          11       0.00      0.00      0.00         3
          12       1.00      0.03      0.05        39
          13       1.00      0.12      0.21        17
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00         5
          16       1.00      0.06      0.12        16
          17       0.17    

D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ShipSupplies\DA\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# import tensorflow as tf

# print("TensorFlow version:", tf.__version__)
# print("Keras version:", tf.keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [11]:
# !pip install gensim

  Using cached gensim-4.3.1-cp310-cp310-win_amd64.whl (24.0 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)


In [102]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 4개 컬럼을 하나의 문자열로 합치기
data = df['Machinery'] + ' ' + df['Assembly'] + ' ' + df['청구품목'] + ' ' + df['Part No.1']
y = df["key2"]

# 문장 토큰화
sentences = [text_to_word_sequence(sentence) for sentence in data]

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4) # 벡터사이즈 100

# 단어 임베딩 활용
embedded_data = []
for sentence in sentences:
    embedded_sentence = [model.wv[word] for word in sentence]
    embedded_data.append(embedded_sentence)

In [103]:
# embedded_data
# 문장들의 길이 확인
sentence_lengths = [len(sentence) for sentence in embedded_data]

# 가장 긴 문장의 길이 확인
max_sequence_length = max(sentence_lengths)

print("가장 긴 문장의 길이:", max_sequence_length)

가장 긴 문장의 길이: 28


In [104]:
# y 값을 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [105]:
X,y

(<20517x11320 sparse matrix of type '<class 'numpy.int64'>'
 	with 221653 stored elements in Compressed Sparse Row format>,
 array([18, 18, 18, ...,  5, 20, 20]))

In [106]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embedded_data, y, test_size=0.2, random_state=42)

In [107]:
# type(X_train)
# print([len(x) for x in X_train])
max_length = max([len(x) for x in X_train])

# 모든 원소들이 동일한 길이로 처리되도록 길이를 조절하거나 자른다.
# X_train_padded = []
# for x in X_train:
#     if len(x) < max_length:
#         x += [0] * (max_length - len(x))  # 0으로 패딩
#     elif len(x) > max_length:
#         x = x[:max_length]  # 일정 길이를 초과하는 부분 자르기
#     X_train_padded.append(x)

X_train_padded = []
for x in X_train:
    padding_shape = (max_length - len(x), 100)  # 패딩할 행렬 모양 확인
    padding_matrix = np.zeros(padding_shape)  # 해당 모양의 0 행렬 생성
    if len(x) < max_length:
        x = np.concatenate((x, padding_matrix), axis=0)  # 0으로 패딩
    elif len(x) > max_length:
        x = x[:max_length]  # 일정 길이를 초과하는 부분을 자르기
    X_train_padded.append(x)

X_train_array = np.array(X_train_padded)

In [108]:
# X_train_array
X_test

X_test_padded = []
for x in X_test:
    padding_shape = (max_length - len(x), 100)  # 패딩할 행렬 모양 확인
    padding_matrix = np.zeros(padding_shape)  # 해당 모양의 0 행렬 생성
    if len(x) < max_length:
        x = np.concatenate((x, padding_matrix), axis=0)  # 0으로 패딩
    elif len(x) > max_length:
        x = x[:max_length]  # 일정 길이를 초과하는 부분을 자르기
    X_test_padded.append(x)
X_test_array = np.array(X_test_padded)

In [109]:
# print('X_train_array shape:', X_train_array.shape)
# print('X_train_array dtype:', X_train_array.dtype)
# print('y_train_array shape:', y_train_array.shape)
# print('y_train_array dtype:', y_train_array.dtype)
print('X_train_array shape:', X_train_array.shape)
print('X_train_array dtype:', X_train_array.dtype)
print('X_test_array shape:', X_test_array.shape)
print('X_test_array dtype:', X_test_array.dtype)
print('y_train shape:', y_train.shape)
print('y_train dtype:', y_train.dtype)

X_train_array shape: (16413, 28, 100)
X_train_array dtype: float64
X_test_array shape: (4104, 28, 100)
X_test_array dtype: float64
y_train shape: (16413,)
y_train dtype: int32


In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd
import tensorflow as tf
from scipy.sparse import coo_matrix
import numpy as np
from scipy.sparse import coo_matrix
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# X_train_array = np.asarray(X_train, dtype=np.float32)

# y_train_array = np.asarray(y_train_array, dtype=np.float32)
y_train_array_encoded = to_categorical(y_train)
y_test_array_encoded = to_categorical(y_test)

# Reshape X_train_array to be 2D
X_train_array = X_train_array.reshape(X_train_array.shape[0], -1)

# X_train과 y_train 데이터를 희소 행렬로 변환
# X_train_sparse = coo_matrix(X_train_array)
# y_train_sparse = coo_matrix(y_train_array)
# 모델 정의
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_array.shape[1],)))
model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='linear'))

# 범주수에 해당하는 뉴런을 추가하고, 활성화 함수를 softmax로 변경합니다.
model.add(Dense(61, activation='softmax'))

# 모델 컴파일 및 학습
# model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_array, y_train_array, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "D:\Shi

In [90]:
y_test

array([54, 33, 24, ..., 51, 25, 40])

In [91]:
# X_test_array
X_test_array = X_test_array.reshape(X_test_array.shape[0], -1)

In [92]:
y_pred = model.predict(X_test_array)

129/129 [==============================] - 0s 3ms/step


In [75]:
y_pred = y_pred.reshape(-1)

In [93]:
y_pred

array([[55.083607],
       [31.186554],
       [21.126211],
       ...,
       [23.491358],
       [30.733864],
       [41.91265 ]], dtype=float32)

In [97]:
rounded_y_pred = np.round(y_pred)
f1 = f1_score(y_test, rounded_y_pred, average='macro')
print("F1 Score:", f1)
# f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
# f1, accuracy

F1 Score: 0.032947391607001296


In [100]:
accuracy = accuracy_score(y_test, rounded_y_pred)

In [101]:
accuracy

0.06115984405458089

In [42]:
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
# Logistic Regression 모델 생성 및 학습
# lr = LogisticRegression(random_state=42, C=14, solver="liblinear", penalty='l1', multi_class="ovr", max_iter=1000)
# lr.fit(X_train, y_train)
# y_pred = model.predict(X_test)

#   # 각종 평가지표 계산
# recall = recall_score(y_test, y_pred, average='macro')
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

KeyboardInterrupt: 

In [ ]:
print("Recall: {:.4f}".format(recall))
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("F1-score: {:.4f}".format(f1))


In [20]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
def print_metrics(y_test, X_test, model) :
  # 테스트 데이터셋으로 예측
  y_pred = model.predict(X_test)

  # 각종 평가지표 계산
  recall = recall_score(y_test, y_pred, average='macro')
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  print("Recall: {:.4f}".format(recall))
  print("Accuracy: {:.4f}".format(accuracy))
  print("Precision: {:.4f}".format(precision))
  print("F1-score: {:.4f}".format(f1))
